In [3]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.preprocessing import LabelEncoder
from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence,text
from sklearn.neighbors import KNeighborsClassifier
from sklearn.cluster import KMeans
from sklearn.metrics import accuracy_score 
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split

In [4]:
data = pd.read_csv("fake_job_postings.csv")

In [5]:
location = data["location"].str.split(",", expand= True, n= 2)
location.columns = ["country", "state", "city"]
data[["country", "state", "city"]] = location
data = data.drop(columns= "location")

In [6]:
salary = data["salary_range"].str.split("-", expand= True, n= 1)
data[["min_salary", "max_salary"]] = salary
data = data.drop(columns= "salary_range")

In [7]:
data = data.fillna("N/A")
data["state"] = data["state"].str.strip().apply(lambda x: "N/A" if x == '' else x)
data["country"] = data["country"].str.strip().apply(lambda x: "N/A" if x == '' else x)
data["city"] = data["city"].str.strip().apply(lambda x: "N/A" if x == '' else x)

In [8]:
for i in ["company_profile", "description", "requirements", "benefits"]:
    data[i] = data[i].str.lower()

In [9]:
data.isnull().sum()

job_id                 0
title                  0
department             0
company_profile        0
description            0
requirements           0
benefits               0
telecommuting          0
has_company_logo       0
has_questions          0
employment_type        0
required_experience    0
required_education     0
industry               0
function               0
fraudulent             0
country                0
state                  0
city                   0
min_salary             0
max_salary             0
dtype: int64

<h2>Prototype Selection with K-Means</h2>

In [10]:
data['text'] = data['title'].str.cat(data[['country', 'state', 'city', 'department','min_salary', 'max_salary','company_profile','description','requirements',
                                    'benefits','employment_type','required_education', 'required_experience', 'industry',
                                    'function']].astype(str), sep=' ')

In [11]:
data['text']=data.text.str.replace(r'\W',' ',regex=True)
data['text']=data.text.str.replace(r'\b\d+','',regex=True)
data['text']=data.text.str.replace(r'\S{20,}',' ',regex=True)
data['text']=data.text.str.replace(r'\s{2,}',' ',regex=True)

In [12]:
stop = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]
data['text'] = data['text'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

In [13]:
num_max = 1000000
max_len = 400

tok = Tokenizer(num_words=num_max)
tok.fit_on_texts(data.text)
non_fraud_matrix = tok.texts_to_matrix(data[data["fraudulent"] == 0].text, mode='tfidf')
fraud_matrix = tok.texts_to_matrix(data[data["fraudulent"] == 1].text, mode='tfidf')

In [14]:
data.fraudulent.value_counts()

0    17014
1      866
Name: fraudulent, dtype: int64

In [ ]:
#Use K-means to select 866 non-fraudulent cases
km = KMeans(n_clusters= 886, random_state= 42).fit(non_fraud_matrix)
km_centers = km.cluster_centers_

In [109]:
#train test split - 0.8, 0.1, 0.1, execute after preprocessing
#X_train, X_test, y_train, y_test = train_test_split(data, data["fraudulent"], test_size= 0.10, random_state= 42, stratify= data["fraudulent"])
#X_tran, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size= 0.11, random_state= 42, stratify= y_train)